In [1]:

train_data_path = "G:/PARAG_S/A1_part2/data/data/train.json"
test_data_path = "G:/PARAG_S/A1_part2/data/data/test.json"
val_data_path = "G:/PARAG_S/A1_part2/data/data/dev.json"
glove_path = "G:/NLP_assignment_2/glove.6B.100d.txt"


In [2]:
import nltk
from nltk.tokenize import word_tokenize
import json
# nltk.download('punkt')

# Loading  training_data from JSON file
with open(train_data_path, 'r') as file:
    data = json.load(file)


#loading validation_data from json file
with open(val_data_path, 'r') as file:
    val_data = json.load(file)



#loading test_data from json file
with open(test_data_path, 'r') as file:
    test_data = json.load(file)


# Iterate through each problem
for problem_data in data:
    problem = problem_data["Problem"]
    linear_formula = problem_data["linear_formula"]
    answer = problem_data["answer"]



from nltk.tokenize import word_tokenize

import re
from nltk.tokenize import word_tokenize

# def custom_tokenize_formula(formula):
#     # Ensure spaces around | and inside parentheses
#     formula = re.sub(r'(\|)', r' \1 ', formula)
#     formula = re.sub(r'(\()', r' \1 ', formula)
#     formula = re.sub(r'(\))', r' \1 ', formula)

#     # Now use word_tokenize
#     tokens = word_tokenize(formula)

#     return tokens

# def tokenize_formula(formula):
#     # Define a regular expression pattern to match tokens
#     pattern = r'\||\(|\)|,|n\d+|const_\d+|\w+'
#     # Find all matches of the pattern in the input formula
#     tokens = re.findall(pattern, formula)
#     return tokens

def tokenize_formula(formula):
    # Split the formula by '|'
    parts = formula.split('|')

    # Tokenize each part
    tokens = ['<sos>']
    for part in parts:
        if part:  
            
            open_parenthesis_index = part.find('(')
            if open_parenthesis_index != -1:
                function = part[:open_parenthesis_index]
                arguments = part[open_parenthesis_index+1:-1]  
                arguments_tokens = arguments.split(',')
                # print(arguments_tokens)
                arguments_token_new=[]
                for tok in arguments_tokens:
                    arguments_token_new.append(tok)
                    arguments_token_new.append(',')
                    
                tokens.extend([function, '('] + arguments_token_new[:-1] + [')'])
            else:
                tokens.append(part)
        if tokens[-1] != '|':
            tokens.append('|')
    tokens.append("<eos>")

    return tokens

# def tokenize_data(fromula):
#     tokenized_data = []
#     for item in data:
#         # Tokenize the problem text
#         # problem_tokens = word_tokenize(item['Problem'].lower())

#         # Tokenize the formula using the updated method
#         formula = item['formula'].lower()
#         formula_tokens = ['<sos>'] + tokenize_formula(formula) + ['<eos>']

#         # Get the answer
#         # answer = item['answer']

#         tokenized_data.append((formula_tokens))
#     return tokenized_data



# tokenized_data = tokenize_data(data)

from collections import Counter
from nltk.tokenize import word_tokenize

def build_vocab(texts, is_formula=False):
    counter = Counter()
    for text in texts:
        if is_formula:
            tokens = tokenize_formula(text.lower()) 
        else:
            tokens = word_tokenize(text.lower())  
        counter.update(tokens)

    # Starting the vocab with special tokens
    vocab = {"<pad>": 0, "<unk>": 1 , "<sos>":2, "<eos>":3 ,'<digit>':4}

    
    for word in counter:
        if word.isdigit():
          continue
        if word not in vocab:
            vocab[word] = len(vocab)

    return vocab, counter



import numpy as np
problem_texts = [item["Problem"].lower() for item in data]
linear_formulas = [item["linear_formula"].lower() for item in data]

# Build vocabularies
problem_vocab, _ = build_vocab(problem_texts)
formula_vocab, _ = build_vocab(linear_formulas, is_formula=True)


# # Adding special tokens
# for vocab in (problem_vocab, formula_vocab):
#     vocab["<pad>"] = 0
#     vocab["<unk>"] = 1

# problem_vocab['<eos>'] = len(problem_vocab)


# formula_vocab = {key: value for key, value in formula_vocab.items() if not key.isdigit()}
# problem_vocab = {key: value for key, value in problem_vocab.items() if not key.isdigit()}

# def reindex_vocab(vocab):
    # """Reindex the vocabulary."""
    # new_vocab = {}
    # new_index = 0
    # for key in vocab.keys():
    #     new_vocab[key] = new_index
    #     new_index += 1
    # return new_vocab

# Reindex vocabularies after filtering out digit keys
# problem_vocab = reindex_vocab(problem_vocab)
# formula_vocab = reindex_vocab(formula_vocab)

# problem_vocab["<digit>"] = len(problem_vocab)

# def text_to_indices(text, vocab):
#     tokens = word_tokenize(text.lower())
#     return [vocab.get(token, vocab["<unk>"]) for token in tokens]

# tokenized_problems = [text_to_indices(problem, problem_vocab) for problem in problem_texts]
# tokenized_formulas = [text_to_indices(formula, formula_vocab) for formula in linear_formulas]


def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings


glove_path = glove_path # Update this path
glove_embeddings = load_glove_embeddings(glove_path)

embedding_dim = 100
# embedding_matrix = np.zeros((len(problem_vocab)-1, embedding_dim))

# for word, idx in problem_vocab.items():
#     embedding_vector = glove_embeddings.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[idx] = embedding_vector














# def load_glove_embeddings(glove_file, word_to_index, embedding_dim=50):
#     embeddings = {}
#     with open(glove_file, 'r', encoding='utf8') as file:
#         for line in file:
#             parts = line.split()
#             word = parts[0]
#             vector = np.array(parts[1:], dtype=np.float32)
#             embeddings[word] = vector

#     # The +1 here ensures there is a row in the matrix for every index in word_to_index
#     embedding_matrix = np.zeros((len(word_to_index)-1, embedding_dim))
#     for word, index in word_to_index.items():
#         if word in embeddings:
#             embedding_matrix[index] = embeddings[word]
#         else:
#             # It's common to initialize embeddings for unknown words with zeros or small random numbers
#             embedding_matrix[index] = np.random.normal(scale=0.6, size=(embedding_dim, ))

#     return embedding_matrix


# Adjust the path to your GloVe file and the embedding dimension you wish to use
# glove_path = "G:/NLP_assignment_2/glove.6B.100d.txt"
# embedding_matrix = load_glove_embeddings(glove_path, vocab, 50)

embedding_dim = 100

def add_digit_and_symbol_tokens_to_vocab(vocab):
    for digit in map(str, range(10)):  # Digits 0-9
        if digit not in vocab:
            vocab[digit] = len(vocab)
    for symbol in ['.', ',']:  
        if symbol not in vocab:
            vocab[symbol] = len(vocab)
    return vocab


problem_vocab = add_digit_and_symbol_tokens_to_vocab(problem_vocab)



def create_embedding_matrix(vocab, embeddings, embedding_dim):
    embedding_matrix = np.zeros((len(vocab), embedding_dim))
    for word, idx in vocab.items():
        if word in embeddings:
            embedding_matrix[idx] = embeddings[word]
        else:
            embedding_matrix[idx] = np.random.rand(embedding_dim)  
    return embedding_matrix



problem_embedding_matrix = create_embedding_matrix(problem_vocab, glove_embeddings, embedding_dim)


def text_to_indices(text, vocab):
    tokens = ['<sos>'] + word_tokenize(text.lower()) + ['<eos>']   # Tokenize the text
    indices = []
    for token in tokens:
        if token.isdigit():  # Check if the token is a digit
            indices.append(vocab.get("<digit>", vocab["<unk>"]))  # Use <digit> token
        else:
            indices.append(vocab.get(token, vocab["<unk>"]))  # Use the token itself
    return indices
# def text_to_indices(text, vocab):
#     # Custom tokenization to handle numbers, decimals, commas, and other punctuation separately
#     tokens = ['<sos>']
#     # Match decimal numbers, integers, words, and specific punctuation characters
#     pattern = r'\d+\.\d+|\d+|[\w]+|[.,!?;%]'
#     for token in re.findall(pattern, text.lower()):
#         if re.match(r'^\d+\.\d+$', token):  # Decimal number
#             # Break down decimal numbers into individual characters, keeping the decimal point
#             tokens.extend([char for char in token])
#         elif token.isdigit():  # Integer
#             # Break down integers into individual digits
#             tokens.extend([char for char in token])
#         else:
#             # For non-numeric tokens, add them directly
#             tokens.append(token)
#     tokens.append('<eos>')
#     # print(tokens)
#     # Convert tokens to indices, using '<unk>' for unknown tokens
#     indices = [vocab.get(token, vocab.get('<unk>', 0)) for token in tokens]

    # return indices
# def tokenize_number(token, vocab):
#     """
#     Breaks down a number into its constituent digits and symbols (., ,).
#     Returns a list of indices corresponding to each digit/symbol in the vocab.
#     """
#     # Symbols to be tokenized separately
#     special_symbols = {'.', ','}
#     indices = []
#     for char in token:
#         if char in special_symbols or char.isdigit():
#             indices.append(vocab.get(char, vocab["<unk>"]))
#         else:
#             raise ValueError("Unexpected character in number: {}".format(char))
#     return indices

# def text_to_indices(text, vocab):
#     tokens = ['<sos>'] + word_tokenize(text.lower()) + ['<eos>']
#     indices = []
#     for token in tokens:
#         if any(char.isdigit() for char in token):  # If token contains any digit
#             indices.extend(tokenize_number(token, vocab))
#         else:
#             indices.append(vocab.get(token, vocab["<unk>"]))
#     return indices



# def text_to_indices(text, vocab):
#     return [vocab.get(token, vocab["<unk>"]) for token in word_tokenize(text.lower()) if not token.isdigit()]
def text_to_indices_formula(text, vocab):
    return [vocab.get(token, vocab["<unk>"]) for token in tokenize_formula(text.lower())]
# def text_to_indices(text, vocab):
#     for token in word_tokenize(text.lower()):
#         if not token.isdigit():
#             return [vocab.get(token, vocab["<unk>"])]





tokenized_problems = [text_to_indices(item["Problem"], problem_vocab) for item in data]
tokenized_formulas = [text_to_indices_formula(item["linear_formula"], formula_vocab) for item in data]

#tokenizing validation data

tokenized_val_problems = [text_to_indices(item["Problem"], problem_vocab) for item in val_data]
tokenized_val_formulas = [text_to_indices_formula(item["linear_formula"], formula_vocab) for item in val_data]


#tokenizing test data
tokenized_test_problems = [text_to_indices(item["Problem"], problem_vocab) for item in test_data]
tokenized_test_formulas = [text_to_indices_formula(item["linear_formula"], formula_vocab) for item in test_data]



from torch.utils.data import Dataset, DataLoader
import torch

class MathProblemDataset(Dataset):
    def __init__(self, problems, formulas):
        self.problems = problems
        self.formulas = formulas

    def __len__(self):
        return len(self.problems)

    def __getitem__(self, idx):
        problem = torch.tensor(self.problems[idx], dtype=torch.long)
        formula = torch.tensor(self.formulas[idx], dtype=torch.long)
        return {"problem": problem, "formula": formula}

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    problems = [item["problem"] for item in batch]
    formulas = [item["formula"] for item in batch]

    problems_padded = pad_sequence(problems, batch_first=True, padding_value=problem_vocab["<pad>"])
    formulas_padded = pad_sequence(formulas, batch_first=True, padding_value=formula_vocab["<pad>"])

    return {"problem": problems_padded, "formula": formulas_padded}

# Initializing the dataset with tokenized and indexed problems and formulas

dataset = MathProblemDataset(tokenized_problems, tokenized_formulas)

val_dataset = MathProblemDataset(tokenized_val_problems, tokenized_val_formulas)

test_dataset = MathProblemDataset(tokenized_test_problems, tokenized_test_formulas)


# Creating DataLoader
data_loader = DataLoader(dataset, batch_size=20 ,shuffle=True, collate_fn=collate_fn)
val_data_loader = DataLoader(val_dataset, batch_size=20, shuffle=True, collate_fn=collate_fn)
test_data_loader = DataLoader(test_dataset, batch_size=20, shuffle=True, collate_fn=collate_fn)


import torch
import torch.nn as nn
import torch.optim as optim
import random
from tqdm import tqdm
import torch.nn.functional as F


class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        print(f'enc dim : {enc_hid_dim} | dec dim : {dec_hid_dim}')
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: [batch_size, dec_hid_dim]
        # encoder_outputs: [src_len, batch_size, enc_hid_dim * 2]
        # print(f'inside attention hidden : {hidden.shape} | encoder_outputs : {encoder_outputs.shape}')
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        # print(f'inside_attention src_len {src_len}')
        # Repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        # print(f'inside attention hidden {hidden.shape}')
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # print(f'inside_attention endoder ouput {encoder_outputs.shape}')

        # print(f'concat {torch.cat((hidden, encoder_outputs), dim=2).shape}')
        # Calculate energy
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))

        # attention: [batch_size, src_len]
        attention = self.v(energy).squeeze(2)

        return F.softmax(attention, dim=1)


class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, embedding_matrix, n_layers=1):
        super().__init__()
        # print(embedding_matrix.shape)
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float),freeze=False)
        self.rnn = nn.LSTM(emb_dim, enc_hid_dim, num_layers=n_layers, bidirectional=True, dropout=dropout if n_layers > 1 else 0)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # print("******************encoder**********************")
        # src shape: [src_len, batch_size]
        # print(f'inside encoder src shape {src.shape}')
        embedded = self.dropout(self.embedding(src))
        # embedded shape: [src_len, batch_size, emb_dim]
        # print(f'inside encoder embedded shape {embedded.shape}')
        # embedded_ = embedded.permute(1,0,2)
        # print(f'embedded_shape {embedded_.shape}')

        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs shape: [src_len, batch_size, enc_hid_dim * num_directions]
        # hidden, cell shape: [num_layers * num_directions, batch_size, enc_hid_dim]
        # print(f'encoder_hidden_shape {hidden.shape} | encoder_cell_shape {cell.shape} | outputs {outputs.shape}')

        # Concatenate the hidden states from both directions
        hidden = torch.cat((hidden[0], hidden[1]), dim=-1)
        cell = torch.cat((cell[0], cell[1]), dim=-1)
        # hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        # cell = torch.tanh(self.fc(torch.cat((cell[-2,:,:], cell[-1,:,:]), dim=1)))
        # print(f"final_encoder_output_hidden {hidden.shape}")
        # print(f"final_encoder_output_cell {cell.shape}")
        # print("***********************************************")

        # print(outputs)
        return outputs, hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM((enc_hid_dim * 2) + emb_dim, dec_hid_dim, dropout=dropout)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(0)  # input: [1, batch_size]
        # print(f'input inside  decoder {input.shape}')
        embedded = self.dropout(self.embedding(input))  # embedded: [1, batch_size, emb_dim]
        # print(f'embedded inside decoder {embedded.shape}')
        # Calculate attention weights
        # print(hidden.shape, encoder_outputs.shape)
        attention = self.attention(hidden, encoder_outputs)  # attention: [batch_size, src_len]

        # Apply attention weights to encoder outputs
        attention = attention.unsqueeze(1)  # attention: [batch_size, 1, src_len]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)  # encoder_outputs: [batch_size, src_len, enc_hid_dim * 2]
        weighted = torch.bmm(attention, encoder_outputs)  # weighted: [batch_size, 1, enc_hid_dim * 2]
        weighted = weighted.permute(1, 0, 2)  # weighted: [1, batch_size, enc_hid_dim * 2]

        rnn_input = torch.cat((embedded, weighted), dim=2)  # rnn_input: [1, batch_size, (enc_hid_dim * 2) + emb_dim]

        output, (hidden, cell) = self.rnn(rnn_input, (hidden.unsqueeze(0), cell.unsqueeze(0)))

        # Prepare input for fully connected layer
        output = output.squeeze(0)  # output: [batch_size, dec_hid_dim]
        embedded = embedded.squeeze(0)  # embedded: [batch_size, emb_dim]
        weighted = weighted.squeeze(0)  # weighted: [batch_size, enc_hid_dim * 2]

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))

        return prediction, hidden.squeeze(0), cell.squeeze(0), attention.squeeze(1)



        # src_ = src.permute(1,0)
        # trg_ = trg.permute(1,0)
        # # print(f'src {src.shape} | src_ {src_.shape}')
        # # print(f'trg {trg.shape} | trg_ {trg_.shape}')
        # batch_size = trg_.shape[1]
        # trg_len = trg_.shape[0]

        # # print(f'batch_size {batch_size} | trg_len {trg_len}')
        # trg_vocab_size = self.decoder.output_dim

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.6):
        src_ = src.permute(1,0)
        trg_ = trg.permute(1,0)
        # print(trg_)
        batch_size = trg_.shape[1]
        trg_len = trg_.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(trg_len-1, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden, cell = self.encoder(src_)

        input = trg_[0,:]
        # input = torch.tensor(2).unsqueeze(0).to(device)
        # print(f'input outside loop {input}')

        for t in range(0, trg_len-1):
            # print(f'input seq to seq {input}')
            output, hidden, cell, _ = self.decoder(input, hidden, cell, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg_[t+1,:] if teacher_force else top1



        return outputs




import torch.optim as optim
import random
import os
from tqdm import tqdm

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


ENC_EMB_DIM = 100  # Embedding dimension for encoder
DEC_EMB_DIM = 100  # Embedding dimension for decoder
ENC_HID_DIM = 256  # Hidden dimension for encoder
DEC_HID_DIM = 512 # Hidden dimension for decoder 
ENC_DROPOUT = 0.5  # Dropout rate for encoder
DEC_DROPOUT = 0.5  # Dropout rate for decoder
OUTPUT_DIM = len(formula_vocab)  # Output dimension is the size of formula vocabulary

# Instantiating the model components
encoder = Encoder(len(problem_vocab), ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT, problem_embedding_matrix)
attention = Attention(ENC_HID_DIM, DEC_HID_DIM)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attention)
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Seq2Seq model
model = Seq2Seq(encoder, decoder, device).to(device)


# Optimizer

learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)


# Loss function, ignoring the index of the padding
PAD_IDX = formula_vocab["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)


def train(model, iterator, optimizer, criterion, clip):
    model.train()  
    epoch_loss = 0

    for i, batch in enumerate(tqdm(iterator)):
        input_ids = batch['problem'].to(device)

        trg = batch['formula'].to(device)
       
        trg_input = trg
        trg_output = trg[:, 1:]
        # trg_new = trg[:, 1:]
        # print(f'trg_out : {trg_output.shape}')
        # print(f'trg_output : {trg_output}')

        optimizer.zero_grad()  

        output = model(input_ids, trg)
        # print(f'output {output.shape}')

        # Reshape output and target tensors to compute loss
        output_dim = output.shape[-1]
        # output = output.contiguous().view(-1, output_dim)
        # trg_output = trg_output.contiguous().view(-1)
        output=output.permute(1,2,0)
        # Calculate loss
        loss = criterion(output, trg_output)
        # print(loss)
        loss.backward() 

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)  # Gradient clipping

        optimizer.step() 
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch['problem'].to(device)
            trg = batch['formula'].to(device)
            # print(src)

            trg_new = trg[:, 1:]  

            output = model(src, trg, 0)  # Turn off teacher forcing
            # print(f'output shape {output.shape}')
            # print(output)
            output_dim = output.shape[-1]
            output = output.permute(1, 2, 0)  

            loss = criterion(output, trg_new)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

enc dim : 256 | dec dim : 512


c:\Users\ANAND\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
c:\Users\ANAND\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
len(problem_vocab)

8326

In [4]:
with open("attn_problem_vocab.json", "w") as json_file:
    json.dump(problem_vocab, json_file, indent=4)

embedding_matrix_list = problem_embedding_matrix.tolist()
with open("sttn_embedding_matrix.json", "w") as json_file:
    json.dump(embedding_matrix_list, json_file, indent=4)

In [5]:
len(formula_vocab)

145

In [3]:




import torch
import json

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=5)
train_losses = []
valid_losses = []

loss_data = {
    'train_losses': train_losses,
    'valid_losses': valid_losses
}

N_EPOCHS = 30
CLIP = 1


best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    if epoch > 30:
        learning_rate = 0.0001
    train_loss = train(model, data_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_data_loader, criterion)

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Val Loss : {valid_loss:.3f}')

    # Update the scheduler
#     scheduler.step(valid_loss)

   
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_losses': train_losses,
            'valid_losses': valid_losses,
            'best_valid_loss': best_valid_loss
        }
        torch.save(checkpoint, f'/kaggle/working/retraained_model_attention_0.6_checkpoint_epoch_{epoch}.pt')

# Save final model
torch.save(model.state_dict(), '/kaggle/working/retrained_attention_model_0.6.pt')

# Save loss data
with open('/kaggle/working/loss_data_model_attention_update_1.json', 'w') as f:
    json.dump(loss_data, f)



100%|██████████| 990/990 [02:46<00:00,  5.94it/s]


Epoch: 01 | Train Loss: 0.936 | Val Loss : 1.212


100%|██████████| 990/990 [02:44<00:00,  6.02it/s]


Epoch: 02 | Train Loss: 0.769 | Val Loss : 1.307


100%|██████████| 990/990 [02:45<00:00,  5.99it/s]


...

Epoch: 28 | Train Loss: 0.146 | Val Loss : 0.604


100%|██████████| 990/990 [02:46<00:00,  5.94it/s]


Epoch: 29 | Train Loss: 0.143 | Val Loss : 0.641


100%|██████████| 990/990 [02:46<00:00,  5.95it/s]


Epoch: 30 | Train Loss: 0.137 | Val Loss : 0.653
